In [25]:
import d2lzh as d2l
from mxnet import gluon, init
from mxnet.gluon import loss as gloss, nn
import mxnet as mx
from mxnet.gluon import data as gdata, loss as gloss, model_zoo, nn
import os
from mxnet import autograd, gluon, init, nd
import time
from mxnet.gluon import HybridBlock
from mxnet.image import  Augmenter
import random
import cv2
import numpy as np
import sys
from PIL import Image




def Q_liang(max_data,min_data):
    int_bits=0
    max_liang=max(abs(max_data),max(min_data))
    for i in range(8):
        if(max_liang<=2**i):
            int_bits=i
            break
    frac_bits=7-int_bits
    return int_bits,frac_bits              

def liang(data):
    b=data.reshape(1,-1)[0]
    min_wt = b.min()[0].asnumpy()
    max_wt = b.max() [0].asnumpy()
    int_bits,frac_bits = Q_liang(max_wt,min_wt)
    print("该层参数为Q%s.%s"%(int_bits,frac_bits))
    data_test=nd.round(data*(2**frac_bits))
    return data_test
    
    
def INT8_liang(data):
    data_INT=liang(data)      #数据量化             
    data_INT=data_INT.astype("int8") #量化 INT8必须
    data_INT=data_INT.astype("float32") #为了python计算
    return data_INT


net = nn.Sequential()
net.add(nn.Conv2D(channels=1, kernel_size=3, activation='relu'),
        nn.MaxPool2D(pool_size=2, strides=2),
        # Dense会默认将(批量大小, 通道, 高, 宽)形状的输入转换成
        # (批量大小, 通道 * 高 * 宽)形状的输入
        nn.Conv2D(channels=4, kernel_size=3, activation='relu'),
        nn.MaxPool2D(pool_size=2, strides=2),
        nn.Dense(8))


net.load_parameters("D:/traint__minst_as")
#采用最大量化

def INT_8_net_4_test_s(net,X,OUT_RSHIFT):
    global ouput1
    ouput_shift=0
    bias_shift=0
    X=X.astype("int8")
    X=X.astype("float32") #为了python计算
    count=0
    for layer in net:
        str_temp=str(layer)
        cneg_name=str_temp.split("(")[0]
        if(cneg_name=="Conv2D")or(cneg_name=="Dense"):
            ouput_shift=OUT_RSHIFT[count]
            ouput_shift=2**ouput_shift
            X=layer(X)
            #print(X.max(),layer)
            X=X/ouput_shift                                   #输出偏移
            X=nd.round(X)
            #print(X.max())
            X=X.astype("int8")
            if(count==1):
                ouput1=X
            X=X.astype("float32")
            count=count+1
        else:
            X=layer(X)
            X=X.astype("int8")
            X=X.astype("float32")
    return X
# X=Image.open("C:\\Users\\胡程畅\\Desktop\\test_picture\\0.jpg")
# X_array=nd.array(X)
# X_array=X_array/255
# X_array=nd.round(X_array*127).reshape((1,1,28,28))

# result=INT_8_net_4(net_int8,X_array)
# result,result.argmax(axis=1)


data_dir  ='D:\mnist_2'
transform_train = gdata.vision.transforms.Compose([
    # 随机对图像裁剪出面积为原图像面积0.08~1倍、且高和宽之比在3/4~4/3的图像，再放缩为高和
    # 宽均为224像素的新图像
    gdata.vision.transforms.Resize(120),
    gdata.vision.transforms.CenterCrop(100),

    # 随机加噪声
    gdata.vision.transforms.ToTensor()
    # 对图像的每个通道做标准化
   ])
batch_size=100
train_ds = gdata.vision.ImageFolderDataset(
         os.path.join(data_dir, 'train2'), flag=0)

train_iter = gdata.DataLoader(train_ds.transform_first(transform_train),
                              batch_size, shuffle=True, last_batch='keep')


BIAS_LSHIFT=[6,7,6]
OUT_RSHIFT =[9,9,8]

net_int8=INT8_net_pranms_Init(net,BIAS_LSHIFT)
def net_Int8_evaluate_accuracy(data_iter, ctx):
    acc_sum, n = 0.00, 0
    for X, y in data_iter:
        y = y.astype('float32')
        y = y.as_in_context(ctx)
        X=nd.round(X*127)
        outputs = INT_8_net_4_test_s(net_int8,X.as_in_context(ctx),OUT_RSHIFT)
        acc_sum += (outputs.argmax(axis=1) == y).sum().copyto(mx.cpu())
        n += y.size
        acc_sum.wait_to_read()
    return acc_sum.asscalar() / n

net_Int8_evaluate_accuracy

该层参数为Q0.7
该层参数为Q0.7
池化层无法量化
该层参数为Q0.7
该层参数为Q0.7
池化层无法量化
该层参数为Q0.7
该层参数为Q0.7


1.0

获取输出偏移值和偏差偏移值

In [20]:
net = nn.Sequential()
net.add(nn.Conv2D(channels=4, kernel_size=3, activation='relu'),
        nn.MaxPool2D(pool_size=2, strides=2),
        # Dense会默认将(批量大小, 通道, 高, 宽)形状的输入转换成
        # (批量大小, 通道 * 高 * 宽)形状的输入
        nn.Conv2D(channels=4, kernel_size=3, activation='relu'),
        nn.MaxPool2D(pool_size=2, strides=2),
        nn.Dense(8))

net.load_parameters("D:/traint__minst_as")
acc1=d2l.evaluate_accuracy(train_iter,net,mx.cpu())
acc1

0.9794871794871794

In [47]:
acc3=str(acc*100)[0:5]+"%"
acc3,acc1

('96.42%', 0.9642324888226528)

In [8]:
d1=ouput1.swapaxes(1,2)
d2=d1.swapaxes(2,3)
d3=d2.reshape((1,-1))[0]

result=nd.array([3,0,0,0,3,0,0,0,3,0,0,0,8,0,4,0,24,0,31,16,30,21,57,24,44,54,28,26,60,47,0,21,51,34,0,2,27,23,0,0,7,7,0,0,3,0,0,0,3,0,0,0,10,0,6,2,27,0,35,18,22,27,65,23,15,55,48,30,0,46,6,39,0,35,0,49,6,18,0,32,21,23,0,2,17,17,0,0,3,0,0,0,7,0,2,0,18,0,35,16,14,24,70,25,6,50,41,27,0,41,1,41,0,22,0,46,0,12,0,55,0,11,0,51,0,13,0,17,14,16,0,0,4,0,0,0,17,0,24,13,8,17,68,21,0,46,43,27,0,37,0,31,0,10,0,28,0,0,0,32,0,0,20,47,0,0,21,56,0,10,0,28,7,19,0,0,11,0,9,4,6,8,60,19,0,27,56,22,0,30,0,19,0,0,0,23,0,0,0,18,0,0,0,10,0,0,54,25,0,2,45,47,0,12,0,33,0,12,0,2,3,3,28,11,0,12,77,23,0,20,8,24,0,0,0,16,0,0,0,13,0,0,0,5,0,0,5,5,0,0,82,14,0,15,36,35,0,11,0,33,0,7,0,9,0,3,44,9,0,5,63,28,0,0,0,29,0,0,0,16,0,0,0,5,12,0,12,7,24,2,36,13,4,27,65,10,0,37,0,20,0,6,0,29,0,0,0,14,0,0,49,7,0,0,55,32,0,0,0,40,4,0,0,22,33,9,13,12,40,30,32,13,26,55,32,15,0,50,3,10,0,9,0,16,0,0,0,21,0,0,0,7,0,0,46,7,0,6,54,36,0,14,0,58,14,34,3,43,36,62,18,27,20,72,10,22,0,55,0,20,0,4,0,21,0,0,0,17,0,0,0,7,0,0,0,0,0,3,33,0,0,11,36,24,0,17,6,60,0,36,0,61,0,45,0,41,0,19,0,30,0,0,0,25,0,0,0,18,0,0,0,7,0,0,0,0,3,0,0,0,0,0,13,0,0,0,17,0,0,0,0,16,0,0,0,30,0,0,0,29,0,0,0,21,0,0,0,12,0,0,0,2,0,0,0,0,3,0,0,0,3,0,0,0,])
start=300
result.min()


[0.]
<NDArray 1 @cpu(0)>

仿真的结果和实际结果有细微偏差，这是由于CMSIS-NN的四舍五入算法有点问题，对于负数其四舍五入逻辑为

小数部分的绝对值大于0.5小数-1，等于0.5不变，这和标准的四舍五入有区别。举个例子

-3.5 CMSIS 四舍五入 -3
     标准  四舍五入 -4
-3.7 CMSIS 四舍五入 -4
     标准  四舍五入 -4
-3.2 CMSIS 四舍五入 -3
     标准  四舍五入 -3
     
可以自行修改 CMSIS的四舍五入但不推荐,因为有Relu激活函数 负数不管什么四舍五入直接归零。